In [31]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dvu
import seaborn as sns
import os
import pandas as pd
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import joblib
dvu.set_style()

# results_dir = '../results_mar25'
results_dir = '../results'
experiment_filename = '../experiments/01_fit_encoding.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
# r = imodelsx.process_results.fill_missing_args_with_default(
    # r, experiment_filename)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 6/6 [00:00<00:00, 682.76it/s]

experiment varied these params: ['feature_space', 'ndelays', 'nboots']


In [32]:
mets = [c for c in r.columns if 'corrs_test_' in c] + \
    [c for c in r.columns if 'corrs_tune_' in c]

In [33]:
d = r.groupby(cols_varied)[mets].mean().round(4)

In [34]:
(
    d
    .style
    .background_gradient(cmap='viridis', axis=0)
    .format(precision=4)
)

In [4]:
mets

['corrs_tune',
 'corrs_tune_mean',
 'corrs_tune_median',
 'corrs_tune_frac>0',
 'corrs_tune_mean_top1_percentile',
 'corrs_tune_mean_top5_percentile']

In [9]:
row = r.iloc[0]
model_params = joblib.load(join(row.save_dir_unique, 'model_params.pkl'))

In [12]:
model_params['alphas_best'].shape

(95556,)

'/home/chansingh/fmri/results/367581222e71c5f54e885907e6016319a6127a52ec28ff74bb34c4d388e63b91'

In [ ]:
mets = []
for k in corrs:
    c = corrs[k]
    met = get_mets(c)
    met['model'] = k
    mets.append(met)
mets = pd.DataFrame.from_dict(mets).round(3)  # .set_index('model')

In [ ]:
mets.style.set_precision(3).background_gradient(axis='index', cmap='plasma')

In [ ]:
# keys_compare = ['eng1000__ndel=4', 'glove__ndel=4', 'bert-10__ndel=4']
m = mets[
    mets.model.str.contains('__ndel=4') & ~mets.model.str.contains(
        '-5') & ~mets.model.str.contains('-20') & ~mets.model.str.contains('-3')
]
m = m.set_index('model').sort_values('mean-top1%', ascending=False)
m.style.set_precision(3).background_gradient(axis='index', cmap='plasma')

In [ ]:
met = 'mean-top98'
ndels = [1, 2, 3, 4, 5, 6]
ngram_sizes = [3, 5, 10, 20]
vals = np.zeros((len(ndels), len(ngram_sizes)))
for i, ndel in enumerate(ndels):
    for j, ngram_size in enumerate(ngram_sizes):
        s = f'bert-{ngram_size}__ndel={ndel}'
        if s in mets.model.values:
            vals[i, j] = mets[mets.model == s][met].values

In [ ]:
sns.heatmap(vals, xticklabels=ngram_sizes,
            yticklabels=ndels, cbar_kws={'label': met})
plt.xlabel('Ngram sizes')
plt.ylabel('Num delays')
plt.show()

In [ ]:
R, C = 1, 3
glove = 'glove__ndel=4'
bert = 'bert-10__ndel=4'
bert_long = 'bert-20__ndel=4'
eng1000 = 'eng1000__ndel=4'


def set_axes():
    plt.plot([-.4, .6], [-.4, .6])
    plt.xlim([0.2, .7])
    plt.ylim([0.2, 0.7])
    plt.ylabel('BERT corr test')


plt.figure(figsize=(C * 3, R * 3))
plt.subplot(R, C, 1)
plt.plot(corrs[bert_long], corrs[bert], '.', alpha=0.4, ms=1)
plt.xlabel('GloVE corr test')
set_axes()

plt.subplot(R, C, 2)
plt.plot(corrs[bert_long], corrs[bert], '.', alpha=0.4, ms=1)
plt.xlabel('Eng1000 corr test')
set_axes()

plt.subplot(R, C, 3)
plt.plot(corrs[bert_long], corrs[bert], '.', alpha=0.4, ms=1)
plt.xlabel('BERT-long corr test')
set_axes()

plt.show()

In [ ]:
plt.hist(corrs[glove])
plt.hist(corrs[bert], alpha=0.8)

In [ ]:
# compare two subjects
corrs_glove_01 = np.load(
    join(results_dir, 'glove', 'UTS01', 'corrs.npz'))['arr_0']
print('01', np.mean(corrs_glove_01), '03', np.mean(corrs_glove))